In [29]:
import pandas as pd 

In [30]:
events = pd.read_csv('C:/Users/Yunmi/Downloads/sampled_events.csv')
products = pd.read_csv('C:/Users/Yunmi/Downloads/sampled_products.csv')
users = pd.read_csv('C:/Users/Yunmi/Downloads/sampled_users.csv')

In [31]:
events_and_products = pd.merge(events, products, how='inner') # item 기준으로 events와 products를 이너조인
df = pd.merge(events_and_products, users, how='inner') # user 기준으로 events_and_products와 users를 이너조인

In [33]:
#날짜형 변환
df['event_timestamp'] = pd.to_datetime(df['event_timestamp'])

In [34]:
# 연도 추출
df['year'] = df['birth_date'].str.slice(start=0, stop=4)
# 연도 컬럼 형변환
df['year'] = df['year'].astype('float')
# 나이 계산
df['age'] = (2022 - df['year'])
# 연도 컬럼 제거
df.drop(['year'], axis=1, inplace=True)
# 생년월일 컬럼 제거
df.drop(['birth_date'], axis=1, inplace=True)
# 나이 결측치를 0으로 바꾸기
df['age'].fillna("0", inplace=True)
# 나이 컬럼 형변환
df['age'] = df['age'].astype('float')
# 마이너스 팔천살부터 열살미만까진 모두 0으로 대체
df.loc[df['age'] < 10,'age'] = 0

In [36]:
df.age.describe()

count   5880407
mean         31
std          11
min           0
25%          26
50%          31
75%          38
max          99
Name: age, dtype: float64

In [38]:
df.gender.value_counts()

F    4376513
M     839641
Name: gender, dtype: int64

In [39]:
# 1. 여성만 필터링
df_f = df.loc[df['gender']=='F']

In [40]:
df_f.gender.value_counts()

F    4376513
Name: gender, dtype: int64

In [41]:
# 2. 나이 20 30 40대만 필터링
df_f_age = df_f[(df_f['age'] >= 20) & (df_f['age'] <= 49)]

In [42]:
df_f_age.age.describe()

count   4124244
mean         33
std           7
min          20
25%          27
50%          31
75%          38
max          49
Name: age, dtype: float64

In [43]:
# 3. 안드로이드만 필터링
df_f_age_aos = df_f_age.loc[df_f_age['platform']=='ANDROID']

In [44]:
df_f_age_aos.platform.describe()

count      999613
unique          1
top       ANDROID
freq       999613
Name: platform, dtype: object

In [45]:
df2 = df_f_age_aos

In [46]:
df.shape, df2.shape

((5880407, 26), (999613, 26))

In [107]:
df2['user_no'].nunique()

33592

# 1. df_groupby 제작
* 장점: 데이터소실이 거의 없음 (원본 데이터셋의 99% 구매건)
* 단점: 분석이 어려움

In [108]:
temp = df2.groupby(['user_no', 'age', 'item_no', 'event_name', 'event_timestamp'], as_index=False)['item_name'].count()
temp

,user_no,age,item_no,event_name,event_timestamp,item_name
0,++1HN9s1imztN8ftPCt3LA==,37,+qoz3h7+OBZNsvUtfx3aeQ==,purchase_success,2021-06-10 09:24:07.981038,1
1,++C4Jco6ku9bc5GwwNuByQ==,48,+YSHYi6S2X00vyqeQ+RDxg==,click_item,2021-06-12 12:10:08.832216,1
2,++C4Jco6ku9bc5GwwNuByQ==,48,/yypsvc4ypxouNQrMSkOFg==,add_to_cart,2021-07-29 13:26:44.885048,1
3,++C4Jco6ku9bc5GwwNuByQ==,48,/yypsvc4ypxouNQrMSkOFg==,purchase_success,2021-07-30 13:36:38.984035,1
4,++C4Jco6ku9bc5GwwNuByQ==,48,0ghoRWEQZgKHfWDWkKfnXw==,add_to_cart,2021-06-12 12:08:15.209044,1
...,...,...,...,...,...,...
996994,zzNoEnoQxBaBTTc79MBmjA==,42,vLyfg01KuFlhdBCVxbfAaw==,click_item,2021-07-05 21:37:39.986061,1
996995,zzuJMnncH5Wrn/GUA7nqTg==,25,EnzKYLzBWAMEcir/4VTccg==,click_item,2021-07-23 19:54:40.659056,1
996996,zzuJMnncH5Wrn/GUA7nqTg==,25,EnzKYLzBWAMEcir/4VTccg==,purchase_success,2021-07-23 20:01:48.630025,1
996997,zzuJMnncH5Wrn/GUA7nqTg==,25,xWJC05Fli+rbPifiT09u7g==,click_item,2021-07-23 19:54:06.457043,1


In [109]:
temp.drop(columns=['item_name'], inplace=True)

In [111]:
temp = pd.get_dummies(temp, columns = ['event_name'])
temp

,user_no,age,item_no,event_timestamp,event_name_add_to_cart,event_name_click_item,event_name_like_item,event_name_purchase_success
0,++1HN9s1imztN8ftPCt3LA==,37,+qoz3h7+OBZNsvUtfx3aeQ==,2021-06-10 09:24:07.981038,0,0,0,1
1,++C4Jco6ku9bc5GwwNuByQ==,48,+YSHYi6S2X00vyqeQ+RDxg==,2021-06-12 12:10:08.832216,0,1,0,0
2,++C4Jco6ku9bc5GwwNuByQ==,48,/yypsvc4ypxouNQrMSkOFg==,2021-07-29 13:26:44.885048,1,0,0,0
3,++C4Jco6ku9bc5GwwNuByQ==,48,/yypsvc4ypxouNQrMSkOFg==,2021-07-30 13:36:38.984035,0,0,0,1
4,++C4Jco6ku9bc5GwwNuByQ==,48,0ghoRWEQZgKHfWDWkKfnXw==,2021-06-12 12:08:15.209044,1,0,0,0
...,...,...,...,...,...,...,...,...
996994,zzNoEnoQxBaBTTc79MBmjA==,42,vLyfg01KuFlhdBCVxbfAaw==,2021-07-05 21:37:39.986061,0,1,0,0
996995,zzuJMnncH5Wrn/GUA7nqTg==,25,EnzKYLzBWAMEcir/4VTccg==,2021-07-23 19:54:40.659056,0,1,0,0
996996,zzuJMnncH5Wrn/GUA7nqTg==,25,EnzKYLzBWAMEcir/4VTccg==,2021-07-23 20:01:48.630025,0,0,0,1
996997,zzuJMnncH5Wrn/GUA7nqTg==,25,xWJC05Fli+rbPifiT09u7g==,2021-07-23 19:54:06.457043,0,1,0,0


In [112]:
temp.to_csv('C:/Users/Yunmi/Downloads/df_group.csv') #99.7만

# 2. df_pivot 제작
* 장점: 분석이 용이함
* 단점: 원본의 77%만 가지고 있음, 타임스탬프의 최소값으로 집계했기에 재구매 고려 불가

In [89]:
df_pivot = pd.pivot_table(df2,
                          index = ['user_no', 'age', 'item_no'], 
                          columns = 'event_name', 
                          values = 'event_timestamp', 
                          aggfunc = 'min')
df_pivot

event_name                                                           add_to_cart  \
user_no                  age item_no                                               
++1HN9s1imztN8ftPCt3LA== 37  +qoz3h7+OBZNsvUtfx3aeQ==                        NaT   
++C4Jco6ku9bc5GwwNuByQ== 48  +YSHYi6S2X00vyqeQ+RDxg==                        NaT   
                             /yypsvc4ypxouNQrMSkOFg== 2021-07-29 13:26:44.885048   
                             0ghoRWEQZgKHfWDWkKfnXw== 2021-06-12 12:08:15.209044   
                             120WzztESpPsiA1ErUYWXQ== 2021-06-12 15:13:49.886047   
...                                                                          ...   
zzNoEnoQxBaBTTc79MBmjA== 42  opS7RSXxnaFWK1rOHd8vLQ==                        NaT   
                             qlZCi1C0JD0Y9jUc72oyXQ==                        NaT   
                             vLyfg01KuFlhdBCVxbfAaw==                        NaT   
zzuJMnncH5Wrn/GUA7nqTg== 25  EnzKYLzBWAMEcir/4VTccg==                        NaT   
                             xWJC05Fli+rbPifiT09u7g==                        NaT   

event_name                                                            click_item  \
user_no                  age item_no                                               
++1HN9s1imztN8ftPCt3LA== 37  +qoz3h7+OBZNsvUtfx3aeQ==                        NaT   
++C4Jco6ku9bc5GwwNuByQ== 48  +YSHYi6S2X00vyqeQ+RDxg== 2021-06-12 12:10:08.832216   
                             /yypsvc4ypxouNQrMSkOFg==                        NaT   
                             0ghoRWEQZgKHfWDWkKfnXw== 2021-06-12 12:07:10.129137   
                             120WzztESpPsiA1ErUYWXQ== 2021-06-12 15:13:41.256138   
...                                                                          ...   
zzNoEnoQxBaBTTc79MBmjA== 42  opS7RSXxnaFWK1rOHd8vLQ== 2021-07-21 23:17:04.380130   
                             qlZCi1C0JD0Y9jUc72oyXQ== 2021-07-27 14:27:48.267103   
                             vLyfg01KuFlhdBCVxbfAaw== 2021-07-05 21:37:39.986061   
zzuJMnncH5Wrn/GUA7nqTg== 25  EnzKYLzBWAMEcir/4VTccg== 2021-07-23 19:54:40.659056   
                             xWJC05Fli+rbPifiT09u7g== 2021-07-23 19:54:06.457043   

event_name                                            like_item  \
user_no                  age item_no                              
++1HN9s1imztN8ftPCt3LA== 37  +qoz3h7+OBZNsvUtfx3aeQ==       NaT   
++C4Jco6ku9bc5GwwNuByQ== 48  +YSHYi6S2X00vyqeQ+RDxg==       NaT   
                             /yypsvc4ypxouNQrMSkOFg==       NaT   
                             0ghoRWEQZgKHfWDWkKfnXw==       NaT   
                             120WzztESpPsiA1ErUYWXQ==       NaT   
...                                                         ...   
zzNoEnoQxBaBTTc79MBmjA== 42  opS7RSXxnaFWK1rOHd8vLQ==       NaT   
                             qlZCi1C0JD0Y9jUc72oyXQ==       NaT   
                             vLyfg01KuFlhdBCVxbfAaw==       NaT   
zzuJMnncH5Wrn/GUA7nqTg== 25  EnzKYLzBWAMEcir/4VTccg==       NaT   
                             xWJC05Fli+rbPifiT09u7g==       NaT   

event_name                                                      purchase_success  
user_no                  age item_no                                              
++1HN9s1imztN8ftPCt3LA== 37  +qoz3h7+OBZNsvUtfx3aeQ== 2021-06-10 09:24:07.981038  
++C4Jco6ku9bc5GwwNuByQ== 48  +YSHYi6S2X00vyqeQ+RDxg==                        NaT  
                             /yypsvc4ypxouNQrMSkOFg== 2021-07-30 13:36:38.984035  
                             0ghoRWEQZgKHfWDWkKfnXw==                        NaT  
                             120WzztESpPsiA1ErUYWXQ==                        NaT  
...                                                                          ...  
zzNoEnoQxBaBTTc79MBmjA== 42  opS7RSXxnaFWK1rOHd8vLQ==                        NaT  
                             qlZCi1C0JD0Y9jUc72oyXQ==                        NaT  
                             vLyfg01KuFlhdBCVxbfAaw==                        NaT  
zzuJMnncH5Wrn/GUA7nqTg== 25  

In [90]:
df_pivot.columns

Index(['add_to_cart', 'click_item', 'like_item', 'purchase_success'], dtype='object', name='event_name')

In [91]:
df_pivot.columns.values

array(['add_to_cart', 'click_item', 'like_item', 'purchase_success'],
      dtype=object)

In [92]:
df_pivot2 = df_pivot.copy()

In [93]:
df_pivot2.columns = df_pivot2.columns.values
df_pivot2.reset_index(inplace=True)

In [94]:
df_pivot2

,user_no,age,item_no,add_to_cart,click_item,like_item,purchase_success
0,++1HN9s1imztN8ftPCt3LA==,37,+qoz3h7+OBZNsvUtfx3aeQ==,NaT,NaT,NaT,2021-06-10 09:24:07.981038
1,++C4Jco6ku9bc5GwwNuByQ==,48,+YSHYi6S2X00vyqeQ+RDxg==,NaT,2021-06-12 12:10:08.832216,NaT,NaT
2,++C4Jco6ku9bc5GwwNuByQ==,48,/yypsvc4ypxouNQrMSkOFg==,2021-07-29 13:26:44.885048,NaT,NaT,2021-07-30 13:36:38.984035
3,++C4Jco6ku9bc5GwwNuByQ==,48,0ghoRWEQZgKHfWDWkKfnXw==,2021-06-12 12:08:15.209044,2021-06-12 12:07:10.129137,NaT,NaT
4,++C4Jco6ku9bc5GwwNuByQ==,48,120WzztESpPsiA1ErUYWXQ==,2021-06-12 15:13:49.886047,2021-06-12 15:13:41.256138,NaT,NaT
...,...,...,...,...,...,...,...
710608,zzNoEnoQxBaBTTc79MBmjA==,42,opS7RSXxnaFWK1rOHd8vLQ==,NaT,2021-07-21 23:17:04.380130,NaT,NaT
710609,zzNoEnoQxBaBTTc79MBmjA==,42,qlZCi1C0JD0Y9jUc72oyXQ==,NaT,2021-07-27 14:27:48.267103,NaT,NaT
710610,zzNoEnoQxBaBTTc79MBmjA==,42,vLyfg01KuFlhdBCVxbfAaw==,NaT,2021-07-05 21:37:39.986061,NaT,NaT
710611,zzuJMnncH5Wrn/GUA7nqTg==,25,EnzKYLzBWAMEcir/4VTccg==,NaT,2021-07-23 19:54:40.659056,NaT,2021-07-23 20:01:48.630025


In [95]:
df_pivot2.columns

Index(['user_no', 'age', 'item_no', 'add_to_cart', 'click_item', 'like_item',
       'purchase_success'],
      dtype='object')

In [96]:
df_pivot2.to_csv('C:/Users/Yunmi/Downloads/df_pivot.csv') #71만